<a href="https://colab.research.google.com/github/dwidian/ColabNotebook/blob/master/StyleGAN2_Faces_Combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Warning!** Please enable GPU from "Runtime", "Change runtime type", set "Hardware Accelerator" to GPU 

In [0]:
!pip install --force tqdm==4.24.0

Clone styllegan2encoder

In [0]:
!git clone https://github.com/robertluxemburg/stylegan2encoder
%cd stylegan2encoder

In [0]:
!mkdir aligned_images raw_images

Put your images into /raw_images/

Let's view our images :



In [0]:
import PIL.Image
img1 = PIL.Image.open('raw_images/Najwa.jpg')
wpercent = (256/float(img1.size[0]))
hsize = int((float(img1.size[1])*float(wpercent)))
img1 = img1.resize((256,hsize), PIL.Image.LANCZOS)
img2 = PIL.Image.open('raw_images/Raisa.jpg')
wpercent = (256/float(img2.size[0]))
hsize = int((float(img2.size[1])*float(wpercent)))
img2 = img2.resize((256,hsize), PIL.Image.LANCZOS)
display(img1,img2)

Now we need to get just the faces, cropped and aligned

In [0]:
!python align_images.py raw_images/ aligned_images/

In [0]:
display(PIL.Image.open('aligned_images/Najwa_01.png').resize((256,256)))
display(PIL.Image.open('aligned_images/Raisa_01.png').resize((256,256)))

In [0]:
%tensorflow_version 1.x

Finally, let's try encoding some images into a latent representation.

In [0]:
!python encode_images.py aligned_images/ generated_images/ latent_representations/ \
    --vgg_url=https://rolux.org/media/stylegan/vgg16_zhang_perceptual.pkl \
    --lr=0.25 --iterations=1000 --use_l1_penalty=0.5

In [0]:
display(PIL.Image.open('generated_images/Najwa_01.png').resize((512, 512)))
display(PIL.Image.open('generated_images/Raisa_01.png').resize((512, 512)))

In [0]:
import dnnlib
import dnnlib.tflib as tflib
import pickle
import PIL.Image
import numpy as np
# load the StyleGAN model into Colab
URL_FFHQ = 'https://drive.google.com/uc?id=1spmHuEW7ie62nXT0qIwFDP7EdAyD112-'
tflib.init_tf()
with dnnlib.util.open_url(URL_FFHQ) as f:
    generator_network, discriminator_network, Gs_network = pickle.load(f)
# load the latents
s1 = np.load('latent_representations/Raisa_01.npy')
s2 = np.load('latent_representations/Najwa_01.npy')
s1 = np.expand_dims(s1,axis=0)
s2 = np.expand_dims(s2,axis=0)
# combine the latents somehow... let's try an average:
savg = 0.5*(s1+s2)
# run the generator network to render the latents:
synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
images = Gs_network.components.synthesis.run(savg, randomize_noise=False, **synthesis_kwargs)
display(PIL.Image.fromarray(images.transpose((0,2,3,1))[0], 'RGB').resize((512,512),PIL.Image.LANCZOS))